In [11]:
import torch
from tuned_lens.nn.lenses import TunedLens
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from tuned_lens.scripts.train_loop_latent import RelatedCollator
from tuned_lens.scripts.lens_latent import RelatedDataset
from tuned_lens.plotting import plot_lens, get_lens_stream
#import ipywidgets as widgets
from plotly import graph_objects as go
import json
import os

In [3]:
import os

import torch
from tuned_lens.nn.lenses import TunedLens, LogitLens
from transformers import AutoModelForCausalLM, AutoTokenizer

windows = False
try:
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  if os.name == 'nt':
    windows = True

bfloat16_available = torch.cuda.get_device_capability()[0] >= 8
floatDtype = torch.bfloat16 if bfloat16_available else torch.float16

In [6]:
device = torch.device('cuda')
loadLens = True
modelName = 'EleutherAI/pythia-1b'
LensName = "..\\outputs\\test-1b\\"
# modelName = 'sshleifer/tiny-gpt2'
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained(modelName, low_cpu_mem_usage=True, torch_dtype=floatDtype)
model.eval()
model.requires_grad_(False)
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained(modelName)
if loadLens:
    tuned_lens = TunedLens.load(LensName, map_location=device)
else:
    tuned_lens = TunedLens(model)
tuned_lens = tuned_lens.to(device)
# logit_lens = LogitLens(model)
logit_lens = None


In [17]:
dataDir = "..\\datasets/"
with open(dataDir +"related.json", "r") as f: 
    relatedJson = json.load(f)
        
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left', return_special_tokens_mask=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

dataset = RelatedDataset(relatedJson, tokenizer)
collator = RelatedCollator(tokenizer, dataset.pad_to_multiple_of)
dl = DataLoader(dataset, batch_size=1, collate_fn=collator, shuffle=True)





In [20]:
for prompt, response, related in dl:
    len_related_words = len(related['input_ids'])

    print("prompt", prompt["input_ids"].shape)
    
    for i in range(len_related_words): 
        hidden_lps, responseOutput = get_lens_stream(model, tokenizer, tuned_lens, 
                                                     input_ids=prompt["input_ids"], 
                                                     input_att_mask=prompt["attention_mask"], 
                                                     response_ids=response["input_ids"], 
                                                     response_att_mask=response["attention_mask"], 
                                                     related_ids = related['input_ids'], 
                                                     related_att_mask=related['attention_mask'])
        
        print(hidden_lps.layers[0].shape)
        print(responseOutput.logits.shape)
    break
    
    # for each word put it in the modified plot lens to make an appropriate return 
    #response_masked_input = response['input_ids'] * response['attention_mask']
   
            

prompt torch.Size([1, 9])
torch.Size([1, 9, 50304])
torch.Size([1, 3, 50304])
